### And now - Week 3 Day 3

## AutoGen Core

Something a little different.

This is agnostic to the underlying Agent framework

You can use AutoGen AgentChat, or you can use something else; it's an Agent interaction framework.

From that point of view, it's positioned similarly to LangGraph.

### The fundamental principle

Autogen Core decouples an agent's logic from how messages are delivered.  
The framework provides a communication infrastructure, along with agent lifecycle, and the agents are responsible for their own work.

The communication infrastructure is called a Runtime.

There are 2 types: **Standalone** and **Distributed**.

Today we will use a standalone runtime: the **SingleThreadedAgentRuntime**, a local embedded agent runtime implementation.

Tomorrow we'll briefly look at a Distributed runtime.


In [1]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_core import SingleThreadedAgentRuntime
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv

load_dotenv(override=True)


True

### First we define our Message object

Whatever structure we want for messages in our Agent framework.

the first thing we do is we define our own object, which is going to be used for passing information around the place. Our own message object. And it is a type of, I don't know if this is required, but a data class means it's a class that's not going to have any methods. All it is is something that holds data. And that makes sense, because the message is not going to be something with functionality. It's going to be used to transport information between our agents. And in a way, doing this is sort of analogous to when we were in LandGraph. We started by defining a state. LandGraph is a state machine that's very focused on what is the state and making sure that that's something that you can move backwards and forwards in. And it's interesting that AutogenCore's fundamental idea is all about messaging. And so the thing that you start by defining is your message. And in our case, we're going to have a very simple one that just has one field, which is content, which is a string. You can experiment with what you pass between your agents, being something much more sophisticated with all sorts of different bits of information. So there we have it. We have a message with a string, and that's all the message we'll be using for today.

In [2]:
# Let's have a simple one!

@dataclass
class Message:
    content: str


### Now we define our Agent

A subclass of RoutedAgent.

Every Agent has an **Agent ID** which has 2 components:  
`agent.id.type` describes the kind of agent it is  
`agent.id.key` gives it its unique identifier

Any method with the `@message_handler` decorated will have the opportunity to receive messages.

So now we get to define our agent in AutogenCore. And it's important to bear in mind that the agent that we're about to create in AutogenCore is different to the agent that we just created in AgentChat. It's difficult. It's like a tongue twister to say all of this. It's hard to keep your mind on it. You're like, what? How is it different? Well, look, the agent that you define in AutogenCore is just like a wrapper. It's like saying, this is a thing that can be messaged. It can be created, it can be managed, it can be referred to, and it can be messaged. But what you do with this is your responsibility. And you're going to have to delegate to something. But this is like a management object, if you will, a management object which you're going to use. And it has, amongst other things, it has an agent ID. And every agent has a unique ID. And that ID has two parts to it, a type and a key. So every single agent has a type and a key. And that combination of type and key is then unique and can uniquely identify it. So if you imagine we'll meet in the future, look at a distributed runtime, in that runtime there could be agents from all over the world collaborating in this distributed runtime. But every one of them will have a unique type plus ID that identifies it precisely. And any agent will be able to talk to another one if it knows its name and its ID. Sorry, its key and its ID. So this gives you a sort of good sense of the fabric of it. And that an agent class that you make is not actually an LLM, it's not an autogen agent chat agent, it's just this management holder object, something which has a type and a key.

**So here is our first autogen core agent. It's called SimpleAgent, as you will see:**


In [3]:
class SimpleAgent(RoutedAgent):
    def __init__(self) -> None:
        super().__init__("Simple")

    @message_handler
    async def on_my_message(self, message: Message, ctx: MessageContext) -> Message:
        return Message(content=f"This is {self.id.type}-{self.id.key}. You said '{message.content}' and I disagree.")
        

It is a subclass of something called `RootedAgent`, which is the typical thing that you have as your parent, as your superclass. And it only has two methods. One of them is the init, the constructor, and all it does is it calls its parent passing its name Simple. That's all it does. Then it has another method called OnMyMessage, which is an async, so it's a coroutine, it's an async method, it is an async coroutine, it is a coroutine. And it's decorated with `@message_handler`. And that's what matters. Any method that you decorate with `@message_handler` means that potentially this is something that will receive messages. And autogen core handles the fact that you'll be able to register this and a runtime is going to manage this. And if someone sends a message to something with my name and my ID, then it's going to end up here. At least, it's going to end up here if what they dispatch is a message of this class. 

```py
@dataclass
class Message:
    content: str
```

So this is a bit of a pro thing, but you can have multiple different classes and you can use that to be able to handle different types of message. 

This may be more detailed than you need, but you could, for example, have a text message and an image message and have two separate handlers. And just by virtue of the different signatures, the different method signatures, autogen core will automatically dispatch the right message to the right method, the right coroutine. That's what it does. It's all about dispatching messages properly. And so this simple agent is going to receive a message and it's going to return a message. Now that I say it, it's a bit like this idea that in LandGraph we took in our nodes, took a state and returned a state. But this is just like a parallel thing, but it's all about messages. It receives a message and it returns a message. And it's an async. It's a coroutine, as I said. So it receives the message, and this one isn't going to do anything with the message it receives. There's no LLM. There's nothing here. It simply returns a message which has, this is, and gives my ID and my key. So it's sort of identifying itself and saying, this is blah, blah, blah. You said, and I'm going to replay back what the person said. So I do actually, I do use this. You said blah, blah, blah, and I disagree. So that is simple agent. Maybe I shouldn't call it a disagreeable agent. That's what it does. There's no LLM. There's no AI involved in this at all. It's just a piece of code. So that doesn't matter to AutoJet Core. AutoJet Core just cares about the handling of messages.

### OK let's create a Standalone runtime and register our agent type

Okay, so now we get to the meat of AutogenCore. We are going to create a runtime, a single-threaded agent runtime, it's called, which is a standalone runtime running on my computer, which, as it says, will handle agents in a single-threaded way. And the first thing that we're going to do is `.register`. We call register on the agent itself to say, I want you to register yourself with this `runtime` (`SimpleAgent.register(runtime`). Now, this isn't creating an agent. This is just saying, you are a type of agent, and I want you to tell the runtime that you are a type of agent that can be spawned, you can be created. And you are a type of agent, a type simple agent, that's going to be your type. And this thing here `SimpleAgent()`, this is a function which can generate new versions of you. It can instantiate you. It is a factory. And you pass that in as well. And so I will do this. 

In [4]:
runtime = SingleThreadedAgentRuntime()
await SimpleAgent.register(runtime, "simple_agent", lambda: SimpleAgent())

AgentType(type='simple_agent')

It's now created a runtime, and it's created a type of agent called a simple agent `AgentType(type='simple_agent')`. We haven't actually yet built any of these. We haven't instantiated one, but as a type of agent, it is now a known thing to our runtime. Our runtime knows that there are such things as simple agents.

### Alright! Let's start a runtime and send a message

And we're now going to start our runtime. It is now running. It is a proper runtime.

In [5]:
runtime.start()

And now we are going to first of all create an agent ID object to identify an agent. And we'll do this properly. We'll say agent ID equals that. That is an ID that would identify this, and we are going to say that we want the `default` agent, because that will then make sure that we have an agent created. 

And then we are going to send a message `send_message` to the agent, which is called simple agent, the default ID, and we're going to send a message, **well hi there!**, to it. And then we'll print whatever comes back. So, any ideas what's going to come back, I wonder? So, it replied, this is simple agent dash default.

In [6]:
agent_id = AgentId("simple_agent", "default")
response = await runtime.send_message(Message("Well hi there!"), agent_id)
print(">>>", response.content)

>>> This is simple_agent-default. You said 'Well hi there!' and I disagree.


So, simple agent is its type, which is the same one we registered, and default is the ID. And it said, you said, well, hi there, and I disagree. 

In [7]:
await runtime.stop()
await runtime.close()

### OK Now let's do something more interesting

We'll use an AgentChat Assistant!

And that is, that's it. So there is a demo of AutogenCore, and that's what AutogenCore does. And putting in agent functionality, like having code in there that calls LLMs, that's your job, or my job. That's not AutogenCore's job. What it's there to do is to handle the passing of messages around by looking things up based on their type and their ID, and that's what it does.

So now I'll show you one which has an LLM behind it, but it shouldn't be any surprise to you because it's just the same infrastructure. The fact that we're calling an LLM is almost of no consequence to the AutogenCore framework. So what we're going to do is we're going to make a slightly more interesting one called MyLLMAgent, still a subclass of RoutedAgent, and we're going to use a proper LLM, and we could just call OpenAI directly using OpenAI.chat.completions.create. We could just call the Python client library. But we can also use AutogenAgentChat, and we might as well since it's AutogenWeek. So this is the same kind of thing, a subclass of RoutedAgent. 

There's going to be two functions again, `init` and `handleMyMessage`. So the ++init++ just calls the superclass `"LLMAgent"`, but it then creates a `model_client`, but this could be doing anything, but this is the AutogenAgentChat way. We're going to create one of these things, a GPT40 mini model client, 

and we're going to set a field called underscore `_delegate`, which we're just sort of holding onto. This is the underlying, this is what our agent object is going to delegate to when it actually needs some code, an LLM to run. And we could have anything we want. We could reply back with random things, or we could create an instance of `AssistantAgent()`, and that's what we're going to do. And that's where we put in, if you remember when you create an assistant agent, there's the name of the agent, and there's the model client, and there it is. And as a result, an underscore delegate, underscore often used to show private secret variables that other people shouldn't know about, like private in the Java world. So this is being set to an assistant agent.

```py
class MyLLMAgent(RoutedAgent):
    def __init__(self) -> None:
        super().__init__("LLMAgent")
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent("LLMAgent", model_client=model_client)
```

So now we just have to write our message handler. `async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:`

And as I say, it can be called anything you like. I'm calling it `handleMyMessageType`. And that's really just to draw your attention to the fact that what matters is what type of object comes in the `message` field, because autogen call will automatically route messages to agents that should receive them based on the finding a handler that looks for a message of that type. That's how it works. It's the clever routing that it does. And by the way, you can send a message direct to an agent, which is what we're doing right now, but it also has a whole kind of pub-sub thing there where you can subscribe to topics and you can publish out messages to lots of agents that are all interested in one particular topic. So the whole process of dispatching messages to the right agent and then calling the right handler, that's the clever stuff. That's what it does well and why we use a framework like this rather than coding it ourselves. But for some of these examples, it's pretty trivial, so we could equally code it ourselves. But you can imagine that this is done in a way that can be scaled and distributed.

So anyways, handleMyMessageType, as long as a message (`message: Message`) is sent to this agent with the ID and type and that the object that's sent is of this type, it will arrive at this function. And I'm going to say that I received a message and print the content. Then I'm going to... Now, this is confusing. This object here, `TextMessage`, looks subtly different to this object here `Message`. Do you know what's going on here? This textMessage is textMessage from the AutoGen... `from autogen_agentchat.messages import TextMessage` It's this thing here. It's the agent chat messages, the textMessage. The same thing we were using yesterday and the day before when we were interacting with agent chat agents. So this is AutoGen agent chat message, which can be a bit confusing and you might... Yeah, there are ways you could get around that by giving it a different... You can say, like, as `AgentChatTextMessage` if you wanted to make it so the code is a little bit clearer and you're distinguishing between the AutoGen call and the AutoGen agent chat. 

All right. So we create the sort of text message that you need for AutoGen agent chat, passing in the content of our message and the source is user. And this is the onMessage that we call in agent chat land with this text message and the castellation token, if you remember that thing. And then we get back the content. We say that that was the reply and we return a new instance of message, of our message, with the content being that reply. I hope you follow all that or at least got enough of an idea of it. Let's give this a shot.


In [8]:

class MyLLMAgent(RoutedAgent):
    def __init__(self) -> None:
        super().__init__("LLMAgent")
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent("LLMAgent", model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        print(f"{self.id.type} received message: {message.content}")
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        reply = response.chat_message.content
        print(f"{self.id.type} responded: {reply}")
        return Message(content=reply)
    


Let's give this a shot. Let's create a new runtime, a single-threaded agent runtime, standalone runtime. We're going to register two agents, the simple agent and the LLM agent, the new one. Simple agent isn't really an LLM at all, it just disagrees with whatever it's told. An LLM agent is actually going to dispatch through this underscore delegate down to true GPT-4 or many. 

In [9]:
from autogen_core import SingleThreadedAgentRuntime

runtime = SingleThreadedAgentRuntime()
await SimpleAgent.register(runtime, "simple_agent", lambda: SimpleAgent())
await MyLLMAgent.register(runtime, "LLMAgent", lambda: MyLLMAgent())

AgentType(type='LLMAgent')

So we're going to start the runtime, then we are going to send a message, hi there, to the real agent. And with whatever it replies, we're going to send that, we're going to forward that on to the other agent, to the simple agent, and see what comes out there. So the LLM agent received, hi there, and it said, hello, how can I assist you today? A classic GPT-4 or many response. And the simple agent said, this is simple agent default. You said, hello, how can I assist you today? And I disagree. And there you go. And with that, we will stop this pantomime.

In [10]:
runtime.start()  # Start processing messages in the background.
response = await runtime.send_message(Message("Hi there!"), AgentId("LLMAgent", "default"))
print(">>>", response.content)
response =  await runtime.send_message(Message(response.content), AgentId("simple_agent", "default"))
print(">>>", response.content)
response = await runtime.send_message(Message(response.content), AgentId("LLMAgent", "default"))

LLMAgent received message: Hi there!
LLMAgent responded: Hello! How can I assist you today?
>>> Hello! How can I assist you today?
>>> This is simple_agent-default. You said 'Hello! How can I assist you today?' and I disagree.
LLMAgent received message: This is simple_agent-default. You said 'Hello! How can I assist you today?' and I disagree.
LLMAgent responded: I appreciate your feedback! How would you like me to assist you?


In [11]:
await runtime.stop()
await runtime.close()

### OK now let's show this at work - let's have 3 agents interact!

Did I just say that I'd stop this pantomime? Well, I take that back. We'll do the pantomime one more time. I just added in another response from the nice LLM agent. So I'm going to have the LLM agent respond to, Hi there! I'm going to have the disagreeable agent disagree, and then I'll put it back to GPT-4 and Mini to see how it handles the disagreement. Let's see how this little conversation goes. Hi there! Hello, how can I assist you today? You said, hello, how can I assist you today? And I disagree. And then GPT-4 and Mini says, I appreciate your feedback. How would you prefer? I greet you or assist you? So if you want to be cruel, you can keep this going. You just have to continually disagree and see how GPT-4 and Mini handles that. And once you've had your fun, then you should do this. This stops the runtime and then closes the runtime, which the Autogen core people insist that we do before we start another. So we will do that and be good to it. Okay, but for one more example, and I'm afraid this time we're not going to have true commercial examples because I think this is just one to show the platform at work and you can take this away and figure out what you'd like to do with it. But for this example, we're going to play rock, paper, scissors between a few agents to show, because I just want to illustrate the fact that this collaboration and this interaction is what it's all about. 

So very simply, we have a `Player1Agent` that is a subclass rooted agent and it has an `OpenAIChatCompletionClient` as its underscore `delegate`. It has an `AssistantAgent` that uses that as the model client. 

And then it has a `handle_my_message_type` that takes messages and returns the response. So this is a super vanilla kind of LLM routing that will just simply respond to the message that is given in the message object. 

```py
class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini", temperature=1.0)
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
```


Okay, we've got a `Player2Agent()`, which is exactly the same thing but with one tiny difference. Can you spot it? Can you spot the difference between those two? Well, the difference is that this has Ollama chat completion client instead of OpenAI chat completion client. So we're going to use Olama. We're going to use a local model and we are going to see which model that we pick. For this one, we're going to pick LLAMA 3.2, the 3 billion variant of LLAMA 3.2. So these are a couple of agents. 

```py
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OllamaChatCompletionClient(model="llama3.2", temperature=1.0)
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
```

In [12]:
from autogen_ext.models.ollama import OllamaChatCompletionClient


class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini", temperature=1.0)
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OllamaChatCompletionClient(model="llama3.2", temperature=1.0)
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)

And now we're going to have a third one, a third agent, and this is going to be a rock, paper, scissors agent. So this one is going to have a little bit more of a, it's going to have an instruction. You're playing rock, paper, scissors. First of all, it comes up with this instruction. You're playing rock, paper, scissors. Respond only with the one word, one of the following. Rock, paper, or scissors. And it puts that together into a message. It looks up the default ID, the default key for player one, that type, and it looks up the default for player two, that type, and it's looking them up. And it then, as part of handling its message, it dispatches off a message to these other agents. So it's just choosing to, it's not choosing, it's like coded it, but it is sending those messages off. And with what comes back with the result, it then puts together a little piece of text called judgment, which says, you are playing in rock, paper, scissors. The players have made these choices. Sorry, you're judging, okay, rock, paper, scissors. Players have made these choices, and then it ends with the who wins, question mark, after slipping in player one's choice and player two's choice. And then that gets dispatched off to my model, which is GPT40Mini, and we return the response. So to summarize, we have a total of three agents that we've defined here, three of these like agent managers, these agent wrappers, they're not real agents, they're delegated to real agents. One is called player one, and it delegates to OpenAI, to GPT40Mini. One is called player two, and it delegates to Llama 3.2. 


And then they don't have any prompts, they just do whatever instructions they're told. And then there's one called judge, and judge very clearly sends an instruction to each of player one and two, saying pick rock or paper or scissors. And then with what comes back, it then judges them and returns the outcome. So there we have a simple setup for a game of rock, paper, scissors. And you can, of course, make this something more interesting, you can have it be something that's entertaining, or you can have it be something that's serious, it's commercial. Like when we're thinking next time in week six, we're going to be building like a financial trading setup. You could imagine this could be something where different agents can interact and can argue about something like whether or not a particular equity is a good investment or a poor investment. You can imagine that's the kind of interactions that could be going on. So whilst we're using this here for a superficial exercise, there's plenty of ways you could imagine any time when you want a sort of autonomy and the ability for different agents to be interacting, you could build that kind of commercial logic into this kind of framework. And that is what Autogencore is for. It's not for playing rock, paper, scissors. But anyways, rock, paper, scissors is the cards you've been dealt, and so we might as well play the let's give it a shot. And quite simply, we create a single-threaded agent runtime, which is the simple kind of local runtime. We register player one, we register player two, and give it the ability to instantiate players one and two. And then we register the judge, the rock, paper, scissors, and have everything ready to go. And we start our runtimes. Registered everything. And this is where we do it. We are going to find our rock, paper, scissors default agent. We are going to say go and set it going. And let's see what happens. Off it runs. Agents are talking. Stuff is happening. And there we go. Player one said rock. Player two said scissors. Player one wins because rock beats scissors. Done. And there you have it. There you have agents being organized and interacting, and agents being discovered. The rock, paper, scissors agent discovered these two agents by name, by identifying them through the sending a message through the runtime. And we saw agents interacting in a game of rock, paper, scissors and being judged, managed by AutoGen Core. And so, again, there are these two types, standalone and distributed. Today we did standalone. Tomorrow we'll do distributed. And I just want to stress again that this is to give you a flavor. It's not necessarily as essential for the building of agents that we're doing mostly in this course. But it's good for you to get this insight and to see it and compare it with the other offerings out there. So with that, that wraps up day three. And as I say, tomorrow we get to distributed. I'll see you then. And welcome to week five, day four. We're going to talk more about AutoGen Core. Did I just say that I'd stop this pantomime? Variant of Lama 3.2. So these, we will do that and be good to it. Okay, but for what? Okay. Okay. Okay. Okay.

In [13]:
JUDGE = "You are judging a game of rock, paper, scissors. The players have made these choices:\n"

class RockPaperScissorsAgent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini", temperature=1.0)
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        instruction = "You are playing rock, paper, scissors. Respond only with the one word, one of the following: rock, paper, or scissors."
        message = Message(content=instruction)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message, inner_1)
        response2 = await self.send_message(message, inner_2)
        result = f"Player 1: {response1.content}\nPlayer 2: {response2.content}\n"
        judgement = f"{JUDGE}{result}Who wins?"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + response.chat_message.content)


In [14]:
runtime = SingleThreadedAgentRuntime()
await Player1Agent.register(runtime, "player1", lambda: Player1Agent("player1"))
await Player2Agent.register(runtime, "player2", lambda: Player2Agent("player2"))
await RockPaperScissorsAgent.register(runtime, "rock_paper_scissors", lambda: RockPaperScissorsAgent("rock_paper_scissors"))
runtime.start()

In [15]:
agent_id = AgentId("rock_paper_scissors", "default")
message = Message(content="go")
response = await runtime.send_message(message, agent_id)
print(response.content)

Player 1: rock
Player 2: rock
In this case, both players chose rock. Since rock ties with rock, there is no winner. The result is a tie. 

TERMINATE


In [16]:
await runtime.stop()
await runtime.close()